In [ ]:
#to see the contents of the cloud storage how the dataset it unzipped
!ls -la /root/.keras/datasets


# Besilesomab and Bonescan Binary Classification

In [ ]:
import PIL
import PIL.Image as Image
import os
import tensorflow as tf
import pathlib
import matplotlib.pyplot as plt
import random
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

# The simplest way is to use google drive because it is well connected together
# This is a simple flow how to train something, methods and programming improvements were let out to be more understanble for NMPs

#PATH to the dataset
MAIN_PATH = "/content/drive/GOOGLE_DRIVE_PATH"
IMG_SIZE = (900, 252)

#dataset path with name
DATASET = "PATH_TO_ZIPPED_DATASET/DATASET_NAME_WITHOUT_ZIP"

#classes in the dataset
# create a folder with subfolder bes and bos with scans and zip them in one file
CLASS_NAMES = ["bes","bos"]

# path to save the logs from tensorboard, Colab deletes all the file after some time
# so on you google drive is can persists for offline analysis
LOG_FILE = "PATH_GOOGLE_DRIVE_WHERE_TO_SAVE_LOGS"


#model save path for the checkpoint
MODEL_PATH = "/content/sample_data/models/save_at_{epoch}_bin.keras"

# this enables to extract from zip into folder
archiveTrain = tf.keras.utils.get_file(origin=f"file://{MAIN_PATH}{DATASET}.zip", extract=True)
dataDirTrain = pathlib.Path(archiveTrain+"/NAME_OF_THE_FILE_WITHOUT_ZIP_SUFFIX");

img_height = 900
img_width = 252
batch_size = 110
number_of_classes = len(CLASS_NAMES)

AUTOTUNE = tf.data.AUTOTUNE
#this creates a nice training and validation dataset, although there is a way to make it manually
train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(dataDirTrain,
                                           labels='inferred',
                                           class_names = CLASS_NAMES,
                                           batch_size=batch_size,
                                           validation_split=0.3,
                                           subset="both",
                                           seed=400,
                                           shuffle=True,
                                           image_size=(900,252))

# verification that the classes are in the dataset
class_names = train_ds.class_names
print("class_name:",class_names)

for image_batch, labels_batch in train_ds:
  print("image",image_batch.shape)
  print("label", labels_batch.shape)
  break

# calculation of initial class_weights
# total_number_of_samples/(number_of_classes*sample_class_count)
def get_class_weights(data_dir,class_names,training_split):
  total_class_count=0
  #to get the full sample count divided by the training set
  for clazz in class_names:
    total_class_count += len(os.listdir(pathlib.Path(os.path.join(dataDirTrain,clazz))))*training_split
  #add each initial class weight to a dictionary
  index = 0
  dict = {}
  for clazz in class_names:
    class_count = len(os.listdir(pathlib.Path(os.path.join(dataDirTrain,clazz))))*training_split
    class_weight = total_class_count/(number_of_classes*class_count)
    dict.update({index:class_weight})
    index += 1
    print("Initial weights:", dict)
  return dict

#sample caount for the exponential decay rate
SAMPLES = 0
#printing of the train dataset tensors based on batch size and sample size calculation
for x, y in train_ds:
  SAMPLES += y.shape[0]
print("number of samples",SAMPLES)

#plotting the training dataset
plt.figure(figsize=(10, 10))
# we take the firt batch from the training dataset -> to see what we are feeding into the CNN
take_files = train_ds.take(1)
for images, labels in take_files:
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

# data augmentation, here the data are augmented for the CNN to use
data_augmentation_layers = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomContrast(0.02), #decresed amount of augmentation
    tf.keras.layers.RandomBrightness(0.02),
    tf.keras.layers.RandomRotation((-0.05,0.05))

])

# data is normalized -> this improves the learning and also memory
normalization_layer = tf.keras.layers.Rescaling(1./255)
# prefetching the data
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

image_batch, labels_batch = next(iter(train_ds))
first_image = image_batch[0]

#plotting the augmentet samples
plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
    for i in range(9):
        augmented_images = data_augmentation_layers(images)
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(np.array(augmented_images[0]).astype("uint8"))
        plt.axis("off")
# size of the kernel to go over the feature map
kernel_size = (3,3)
pool_size= (2,2)

def simple_cnn2():
  inputs = tf.keras.Input(shape=(900, 252, 3))
  #this means we add the layers to the model -> makes it more CPU/GPU consuming, but
  # you do not need to preprocess the data for inference, e.g. in your app
  x=data_augmentation_layers(inputs)
  x=normalization_layer(x)
  x= tf.keras.layers.Conv2D(32,kernel_size,padding="same",activation='relu')(x)
  #x= tf.keras.layers.BatchNormalization()(x)
  #x= tf.keras.layers.Conv2D(32,kernel_size,padding="same",activation='relu')(x)
  x=tf.keras.layers.MaxPooling2D(pool_size)(x)
  x=tf.keras.layers.Conv2D(64,kernel_size,padding="same",activation='relu')(x)
  #x= tf.keras.layers.BatchNormalization()(x)
  #x=tf.keras.layers.Conv2D(64,kernel_size,padding="same",activation='relu')(x)
  x=tf.keras.layers.MaxPooling2D(pool_size)(x)
  x=tf.keras.layers.Conv2D(128,kernel_size,padding="same",activation='relu')(x)
  #x=tf.keras.layers.Conv2D(128,kernel_size,padding="same",activation='relu')(x)
  x=tf.keras.layers.MaxPooling2D(pool_size)(x)
  x=tf.keras.layers.Conv2D(512,kernel_size,padding="same",activation='relu')(x)
  #x=tf.keras.layers.Conv2D(512,kernel_size,padding="same",activation='relu')(x)
  x=tf.keras.layers.MaxPooling2D(pool_size)(x)
  x=tf.keras.layers.Flatten()(x)
  #x=tf.keras.layers.Dense(512, activation='relu')(x)
  #x=tf.keras.layers.Dropout(0.5)(x)
  x=tf.keras.layers.Dense(256, activation='relu')(x)
  x=tf.keras.layers.Dropout(0.5)(x) # this is for decreasing the overfitting, some neurons are dropped in the CNN, https://www.tensorflow.org/tutorials/keras/overfit_and_underfit
  x=tf.keras.layers.Dense(128, activation='relu')(x)
  outputs=tf.keras.layers.Dense(1, activation=None)(x)
  return tf.keras.Model(inputs=inputs, outputs=outputs)

model = simple_cnn2()

epochs = 60
# exponential decay rate based on batch size, subsequent stops and epochs
initial_learning_rate = 0.0003
final_learning_rate = 0.00001
learning_rate_decay_factor = (final_learning_rate / initial_learning_rate)**(1/epochs)
steps_per_epoch = int(SAMPLES/batch_size)
print("steps per epoch:",steps_per_epoch)
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
                initial_learning_rate=initial_learning_rate,
                decay_steps=steps_per_epoch,
                decay_rate=learning_rate_decay_factor,
                staircase=True)


model.compile(
              # switch between stable learning rate and scheduler
              optimizer=tf.keras.optimizers.Adam(3e-4),
              #optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[ #metrics
                        tf.keras.metrics.AUC(name="auc"),
                        tf.keras.metrics.TruePositives(name='tp'),
                        tf.keras.metrics.FalsePositives(name='fp'),
                        tf.keras.metrics.TrueNegatives(name='tn'),
                        tf.keras.metrics.FalseNegatives(name='fn'),
                        tf.keras.metrics.Precision(name='precision'),
                        tf.keras.metrics.Recall(name='recall'),
                        keras.metrics.BinaryAccuracy(name="acuraccy")],
              run_eagerly=False
              )
# the training/fit process can be manipulated so weights can be adjusted etc this is the simplest use for the callbacks
callbacks = [
    # this is logging the training process
    tf.keras.callbacks.TensorBoard(log_dir=f"/content/drive/MyDrive/Colab/{LOG_FILE}",write_graph=False),
    # this saves the best models
    tf.keras.callbacks.ModelCheckpoint(MODEL_PATH, monitor="val_loss"),
    # this stops when the model begins to overfit
    #tf.keras.callbacks.EarlyStopping(patience=5, start_from_epoch=35,restore_best_weights=True)
]

model.summary()
#training function
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
  # the callback defined above
  callbacks=callbacks,
  # initial weights to use for the learning phase
  class_weight=get_class_weights(dataDirTrain,CLASS_NAMES,0.7)
)



In [ ]:
#it is better to use Colabs cloud storage for saving best models and then save them to your Google Drive
model = tf.keras.models.load_model("/content/sample_data/models/CHOOSE_THE_BEST_MODEL.keras")
model.summary()
model.save('/content/drive/MyDrive/PATH_TO_YOUR_GOOGLE_DRIVE/MODEL_NAME.keras')

In [ ]:
# this enables to see the log output for the training process
%load_ext tensorboard
#%reload_ext tensorboard #in case you have it already running
%tensorboard --logdir 'PATH_TO_YOUR_GOOGLE_DRIVE_WITH_THE_LOGS'